In [105]:
import pyaudio
import numpy as np
import matplotlib.pyplot as plt
from tkinter import TclError
import struct
import time
from scipy.io.wavfile import write
# use this backend to display in separate Tk window
%matplotlib tk

In [106]:
FORMAT = pyaudio.paInt16 # 16bit format per sample
RATE = 44100 # Samples per second
CHANNELS = 1 # Only one source , your microphone
CHUNK = 1024*2 # We split our data into 1024 samples each time , for utilizing processing power

In [107]:
class Record:
    def __init__(self,forma,rate,chunk,channels):
        try:
            # Initiate details
            self.chunk = chunk
            self.format = forma
            self.channels = channels
            self.rate = rate
            
            self.audio = pyaudio.PyAudio()
            self.stream = self.audio.open(format = self.format,
                                          channels = self.channels,
                                          rate = self.rate, 
                                          input=True)
        except:
            print('Initiation Error')
    
    
    
    def make_plots(self):
        self.fig, (self.ax ,self.ax1) = plt.subplots(2,figsize=(15, 10))
       
        ## FIRST PLOT - FFT
        
        self.ax.set_xlabel('Frequency (Hz)')
        self.ax.set_ylabel('Power (dB)')
        self.ax.set_title('philorfa')
        

        af = np.linspace(0, self.rate, self.chunk//2 +1)                 # set xdimendion
        self.line, = self.ax.plot(af, np.random.rand(self.chunk//2 +1)) # initiate xdimension with random values , will be refreshed
                                                               # with the right values 
        
        self.ax.set_ylim(0, 100)  # set max value for ydimension
        self.ax.set_xlim(0, 10000) # set max value for xdimension
        
        ## SECOND PLOT - SOUNDWAVE
       
        self.ax1.set_xlabel('Samples')
        self.ax1.set_ylabel('Volume')
        
        a = np.arange(0,self.chunk,1) # set x dimension
        self.line1, = self.ax1.plot(a,np.random.rand(self.chunk)) # set ydimension 
        self.ax1.set_ylim(0, 500) 
        self.ax1.set_xlim(0, self.chunk)
        
    def start_streaming_ploting(self):
        
        DFFT=[]
        print("3...")
        time.sleep(1)
        print("2...")
        time.sleep(1)
        print("1...")
        time.sleep(1)
        print("Sream Started")
        
        self.stream.start_stream() # αρχίζει να ακούει 
        
        while True:
            in_data=self.stream.read(self.chunk) # Για την επεξεργασία , χρησιμοποιούμε ενα CHUNK την φορά

            audio_data = np.frombuffer(in_data, np.int16) # Ετοιμο απο τον άλλον κώδικα
            dfft=10*np.log10(abs(np.fft.rfft(audio_data))) # Έτοιμο από τον άλλον κώδικα !!
                                                           # Tο μέγεθος του είναι πίνακας 513 στοιχείων
            xf = np.fft.rfftfreq(audio_data.size, d=1.0/self.rate)
            
            
            
            data_int = struct.unpack(str(2 * self.chunk) + 'B', in_data)  # Μετατρεπω σε int τα bit
            data_np = np.array(data_int, dtype='b')[::2] +128        # Τα μετατρέπω σε np array
            
            DFFT.append(dfft)
            xmid = np.argmax(dfft) # Βρίσκω το index του max
            x=xf[xmid]
            y = np.amax(dfft) # Βρίσκω την τιμη του max
            
            text= "x={:.3f}, y={:.3f}".format(x, y) # Κείμενο στο βελάκι του max
            
            an=self.ax.annotate(text,
            xy=(x, y), xycoords='data',
            xytext=(-15, 25), textcoords='offset points',
            arrowprops=dict(facecolor='black', shrink=0.05),
            horizontalalignment='left', verticalalignment='bottom') # Δημιουργώ το βέλος
    
            self.line.set_data(xf,dfft) # Τοποθετώ τα δεδομένα στο plot
            self.line1.set_ydata(data_np)
            try:
                self.fig.canvas.draw() # Σχεδιάζω
                #time.sleep(0.5) # Παγώνω στιγμιαία
                self.fig.canvas.flush_events() # Καθαρίζω τα δεδομένα
                an.remove() # Καθαρίζω το βέλος
            except TclError:
                print("DONE")
                DFFT=np.array(DFFT) #Μετατρέπω σε np array γιατι αυτό το format θέλει για να αποθηκευτεί
                self.stream.stop_stream() # Σταματάω το stream
                self.stream.close()
                write("fft.wav", self.rate, DFFT) # εξάγω τον integer array σαν wav αρχείο 
                                                 
                break

In [108]:
if __name__ == '__main__':
    
    sound=Record(FORMAT,RATE,CHUNK,CHANNELS) # Create object
    sound.make_plots()  # initiate plot
    sound.start_streaming_ploting() # start stream and

3...
2...
1...
Sream Started
DONE
